In [27]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops.layers.torch import Rearrange


class MultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.0):
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        assert (
            self.head_dim * num_heads == embed_dim
        ), "embed_dim must be divisible by num_heads"

        self.qkv_proj = nn.Linear(embed_dim, embed_dim * 3)
        self.dropout = nn.Dropout(p=dropout)
        self.proj_out = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, L, D = x.size()  # Batch size, sequence length, embedding dimension

        # Project queries, keys, and values
        qkv = self.qkv_proj(x)
        q, k, v = torch.chunk(qkv, 3, dim=-1)

        # Reshape q, k, v to have multiple heads
        q = q.view(B, L, self.num_heads, self.head_dim).transpose(
            1, 2
        )  # B x num_heads x L x head_dim
        k = k.view(B, L, self.num_heads, self.head_dim).transpose(
            1, 2
        )  # B x num_heads x L x head_dim
        v = v.view(B, L, self.num_heads, self.head_dim).transpose(
            1, 2
        )  # B x num_heads x L x head_dim

        # Compute attention scores
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (
            self.head_dim**0.5
        )  # B x num_heads x L x L

        # Apply attention mask (if needed)
        attn_weights = F.softmax(attn_scores, dim=-1)  # B x num_heads x L x L

        # Apply dropout
        attn_weights = self.dropout(attn_weights)

        # Weighted sum of values
        attn_output = torch.matmul(attn_weights, v)  # B x num_heads x L x head_dim

        # Concatenate heads and project out
        attn_output = (
            attn_output.transpose(1, 2).contiguous().view(B, L, self.embed_dim)
        )  # B x L x embed_dim
        attn_output = self.proj_out(attn_output)  # B x L x embed_dim

        return attn_output

In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange, einsum
from typing import Optional
from torch import Tensor

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, nheads, bias=True, dropout=0.0):
        super().__init__()
        assert d_model % nheads == 0, "d_model must be divisible by nheads"

        self.d_model = d_model
        self.nheads = nheads
        self.d_h = d_model // nheads
        self.scaling = self.d_h ** -0.5

        self.wq = nn.Linear(d_model, d_model, bias)
        self.wk = nn.Linear(d_model, d_model, bias)
        self.wv = nn.Linear(d_model, d_model, bias)
        self.wo = nn.Linear(d_model, d_model, bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_bias: Optional[Tensor] = None, mask=None, need_weights=False):

        q = rearrange(self.wq(x), 'b n (h d) -> b h n d', h=self.nheads)
        k = rearrange(self.wk(x), 'b n (h d) -> b h n d', h=self.nheads)
        v = rearrange(self.wv(x), 'b n (h d) -> b h n d', h=self.nheads)
        # Scaled Dot Product Attention
        attn_weights = einsum( q, k, "b h q d, b h k d -> b h q k") * self.scaling
        if attn_bias is not None:
            attn_weights += rearrange(attn_bias, "b i j -> b () i j")
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.dropout(attn_weights)
        attn_output = einsum( attn_weights, v, "b n i j, b h j d -> b h i d")
        attn_output = rearrange(attn_output, "b h n d -> b n (h d)")
        attn_output = self.wo(attn_output)
        if need_weights:
            return attn_output, attn_weights
        else:
            return attn_output


class GroupedQueryAttention(nn.Module):
    def __init__(self, d_model, qheads, kvheads, bias=True, dropout=0.0):
        super().__init__()
        assert d_model % qheads == 0, "d_model must be divisible by nheads"
        assert qheads % kvheads == 0, "qheads must be divisible by kvheads"

        self.d_model = d_model
        self.qheads = qheads
        self.kvheads = kvheads

        self.d_q = d_model // qheads
        d_kv = d_model // qheads * kvheads
        self.scaling = self.d_q**-0.5
        self.num_head_groups = qheads // kvheads

        self.wq = nn.Linear(d_model, d_model, bias)
        self.wk = nn.Linear(d_model, d_kv, bias)
        self.wv = nn.Linear(d_model, d_kv, bias)
        self.wo = nn.Linear(d_model, d_model, bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_bias: Optional[Tensor] = None, mask=None, need_weights=False):

        q = rearrange(self.wq(x), "b n (h d) -> b h n d", h=self.qheads)
        k = rearrange(self.wk(x), "b s (h d) -> b h s d", h=self.kvheads)
        v = rearrange(self.wv(x), "b s (h d) -> b h s d", h=self.kvheads)
        # Grouped Query Attention
        q = rearrange(q, "b (h g) n d -> b g h n d", g=self.num_head_groups)
        attn_weights = einsum(q, k, "b g h n d, b h s d -> b g h n s") * self.scaling
        if attn_bias is not None:
            attn_weights += rearrange(attn_bias, "b i j -> b () () i j")
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.dropout(attn_weights)
        attn_output = einsum(attn_weights, v, "b g h n s, b h s d -> b g h n d")
        attn_output = rearrange(attn_output, "b g h n d -> b n (h g d)")
        attn_output = self.wo(attn_output)
        if need_weights:
            attn_weights = rearrange(attn_weights, "b g h n s -> b n s (h g)")
            return attn_output, attn_weights
        else:
            return attn_output

In [48]:
batch_size = 4
seq_length = 10
embed_dim = 64
num_heads = 8

# Create input tensor
input_tensor = torch.randn(batch_size, seq_length, embed_dim)

# Initialize Multihead Attention layer
multihead_attn = GroupedQueryAttention(embed_dim, 8, 4)

# Forward pass
output_tensor = multihead_attn(input_tensor)

# Print output shape
# print("Output shape:", output_tensor.shape)

In [49]:
output_tensor[0]

tensor([[ 1.2270e-02, -1.1735e-01, -6.7048e-02, -2.4043e-01, -8.7736e-02,
         -4.5804e-02, -1.9562e-02, -2.4857e-01,  1.4149e-01,  3.9893e-02,
          1.3840e-01, -2.0862e-02,  7.3982e-02,  7.8651e-02,  5.7386e-02,
          6.3551e-02, -3.4428e-02, -7.2068e-02,  1.3942e-01, -9.8994e-02,
         -2.1609e-01, -2.1194e-01,  9.7416e-02, -8.3808e-02,  5.5601e-02,
         -1.3729e-01,  1.3815e-01, -6.4847e-02,  1.0566e-01, -2.4299e-02,
         -6.5047e-02, -1.8546e-01, -1.1985e-01, -3.0995e-01, -6.6507e-02,
          8.6111e-02,  5.8910e-02, -5.2422e-02, -2.6515e-02, -3.3529e-02,
          3.0535e-02, -6.8945e-02, -1.8473e-01,  7.3203e-03,  5.1725e-03,
          2.7353e-01, -8.3754e-02,  2.5106e-02,  2.5990e-01,  2.7832e-02,
         -9.1695e-03,  2.1049e-01,  2.3586e-01, -1.2316e-01,  8.9196e-02,
          2.8696e-02, -1.4621e-01,  2.8020e-01,  9.1085e-02, -1.4379e-02,
          1.7279e-01, -4.9902e-02, -1.3725e-01,  9.8382e-02],
        [ 9.2150e-03, -1.5714e-01,  2.9604e-03, -2

In [8]:
embed_dim = embed_dim
num_heads = num_heads
head_dim = embed_dim // num_heads

assert (
    head_dim * num_heads == embed_dim
), "embed_dim must be divisible by num_heads"

qkv_proj = nn.Linear(embed_dim, embed_dim * 3)
dropout = nn.Dropout(p=0)
proj_out = nn.Linear(embed_dim, embed_dim)

In [9]:
qkv_proj

Linear(in_features=64, out_features=192, bias=True)

In [10]:
input_tensor.size()

torch.Size([4, 10, 64])

In [20]:
x = input_tensor
B, L, D = x.size()  # Batch size, sequence length, embedding dimension

d_model = D
wq = nn.Linear(d_model, d_model)
wk = nn.Linear(d_model, d_model)
wv = nn.Linear(d_model, d_model)
# Project queries, keys, and values
q = wq(x)
k = wk(x)
v = wv(x)
# qkv = qkv_proj(x)
# q, k, v = torch.chunk(qkv, 3, dim=-1)



In [21]:
q.size()

torch.Size([4, 10, 64])

In [25]:
from einops import rearrange
q = rearrange(wq(x), 'b n (h d) -> b h n d', h=num_heads)
k = rearrange(wk(x), 'b n (h d) -> b h n d', h=num_heads)
v = rearrange(wv(x), 'b n (h d) -> b h n d', h=num_heads)

In [27]:
q.shape

torch.Size([4, 8, 10, 8])

In [18]:
q = q.view(B, L, num_heads, head_dim).transpose(1, 2)

In [19]:
q.size()

torch.Size([4, 8, 10, 8])

In [30]:
torch.einsum('b h i d, b h j d -> b h i j', q, k).shape

torch.Size([4, 8, 10, 10])

In [42]:
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (head_dim**0.5)

In [43]:
attn_scores.shape

torch.Size([4, 8, 10, 10])

In [34]:
attn_weights = F.softmax(attn_scores, dim=-1)

In [37]:
x = torch.einsum("b h i j, b h j d -> b h i d", attn_weights, v)

In [38]:
x.shape

torch.Size([4, 8, 10, 8])

In [39]:
attn_output = torch.matmul(attn_weights, v)
attn_output.shape

torch.Size([4, 8, 10, 8])

In [41]:
rearrange(attn_output, 'b h n d -> b n (h d)').shape

torch.Size([4, 10, 64])

In [4]:
!conda install -c pyg pyg

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::mkl==2021.2.0=h06a4308_296
  - defaults/linux-64::mkl-service==2.3.0=py38h27cfd23_1
  - defaults/linux-64::watchdog==1.0.2=py38h06a4308_1
  - defaults/noarch::dask-core==2021.4.0=pyhd3eb1b0_0
  - defaults/linux-64::numpy-base==1.20.1=py38h7d8b39e_0
  - defaults/noarch::conda-verify==3.4.2=py_1
  - defaults/linux-64::gevent==21.1.2=py38h27cfd23_1
  - defaults/noarch::jupyter_console==6.4.0=pyhd3eb1b0_0
  - defaults/noarch::nbclient==0.5.3=pyhd3eb1b0_0
  - defaults/noarch::qtconsole==5.0.3=pyhd3eb1b0_0
  - defaults/linux-64::spyder-kernels==1.10.2=py38h06a4308_0
  - defaults/linux-64::anaconda-navigator==2.0.3=py38_0
  - defaults/noarch::nbclassic==0.2.6=pyhd3eb1b0_0
  - defaults/linux-64::spyder==4.2.5=py38h06a4308_0
  - defaults/linux-64::widgetsnbextension==3.5.1=py38_0
  - defaults/noarch::ipywidgets==7.6.3=pyhd

In [4]:
import torch
from torch_geometric.data import Data
from einops.layers.torch import Rearrange

d_model = 128
num_heads = 8
d_ff = 512
num_atoms = 10
num_bonds = 20

x = torch.randn(num_atoms, d_model)
edge_index = torch.randint(0, num_atoms, (2, num_bonds))
edge_attr = torch.randn(num_bonds, d_model)
mask = torch.ones(num_atoms, num_atoms)
mask = mask.masked_fill(torch.eye(num_atoms).bool(), 0)

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, mask=mask)

block = GraphomerBlock(d_model, num_heads, d_ff)
block(data)

NameError: name 'GraphomerBlock' is not defined

In [6]:
from graphomer.datamodule import GraphFeaturizer
from graphomer.model.embeddings import GraphEmbedding
from graphomer.datamodule.featurizer.features import (
    ATOM_FEATURES_DIM,
    BOND_FEATURES_DIM,
)

featurizer = GraphFeaturizer()
atom_features_dim = list(ATOM_FEATURES_DIM.values())
bond_features_dim = list(BOND_FEATURES_DIM.values())
graph_encoder = GraphEmbedding(
    d_model=128,
    atom_features_dim=atom_features_dim,
    bond_features_dim=bond_features_dim,
)


In [21]:
from torch_geometric.utils import to_dense_adj, to_dense_batch
from graphomer.datamodule.featurizer.featurizer import GraphFeaturizer

smi1 = 'c1ccccc1'
smi2 = 'c1cccc(CCCCC)c1'
feat = GraphFeaturizer()
data1 = feat(smi1)
data2 = feat(smi2)
from torch_geometric.data import Data, Batch

batch = Batch.from_data_list((data1, data2))

In [8]:
adj_matrix = to_dense_adj(batch.edge_index, batch.batch, max_num_nodes=40)

In [9]:
import numpy as np
import torch
degree_centrality = torch.sum(adj_matrix, axis=1) / (len(adj_matrix) - 1)

In [10]:
degree_centrality

tensor([[2., 2., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [2., 2., 2., 2., 3., 2., 2., 2., 2., 1., 2., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [1]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj
from typing import List, Optional
from graphomer.datamodule.featurizer.features import (
    ATOM_FEATURES_DIM,
    BOND_FEATURES_DIM,
)
from torch import Tensor
from graphomer.model.embeddings import DiscreteEmbedding, ContinuousEmbedding
class GraphEmbedding(nn.Module):
    def __init__(
        self,
        d_model: int = 768,
        atom_features_dim: List[int] = None,
        bond_features_dim: List[int] = None,
        max_in_degree: int = 512,
        max_out_degree: int = 512,
        cont_features_dim: Optional[List] = None,
    ):
        super().__init__()
        self.atom_embedding = DiscreteEmbedding(d_model, atom_features_dim)
        self.bond_embedding = DiscreteEmbedding(d_model, bond_features_dim)
        # Centrality embedding
        self.z_in = DiscreteEmbedding(d_model, [max_in_degree, max_out_degree])
        self.z_out = DiscreteEmbedding(d_model, [max_out_degree])

        # if cont_features_dim is not None:
        #     self.continuous_embedding = ContinuousEmbedding(d_model, cont_features_dim)

    def forward(self, data: Data):
        print(self.atom_embedding.shape)
        data.x = self.atom_embedding(data.x)
        data.edge_attr = self.bond_embedding(data.edge_attr)
        if self.continuous_embedding:
            data.continuous_attr = self.continuous_embedding(data.continuous_attr)
        adj_matrix = to_dense_adj(data.edge_index, data.batch)
        z_in = self.z_in(adj_matrix.sum(dim=-1))
        z_out = self.z_out(adj_matrix.sum(dim=-2))
        data.x = data.x + z_in + z_out

        return data

from torch_geometric.utils import to_dense_adj, to_dense_batch
from graphomer.datamodule.featurizer.featurizer import GraphFeaturizer

smi1 = "CC(C)(C)OC(=O)N1CCN(CC2=CC=CC=C2)CC1CCO"
smi2 = "CC(C)(O)C1=NC(OC2=CC=C3N(C4=CC(C5=CC=CO5)=NC(N)=N4)N=NC3=C2)=CC=C1"
feat = GraphFeaturizer()
data1 = feat(smi1)
data2 = feat(smi2)
from torch_geometric.data import Data, Batch

batch = Batch.from_data_list((data1, data2))
from graphomer.model.embeddings import GraphEmbedding

graph_encoder = GraphEmbedding(
    atom_features_dim=list(ATOM_FEATURES_DIM.values()), bond_features_dim=list(BOND_FEATURES_DIM.values())
)

/db2/users/jungwookim/projects/personal/graphomer_pyg/graphomer/datamodule/featurizer/featurizer.py:96: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1711403408687/work/torch/csrc/utils/tensor_new.cpp:275.)
  x = torch.tensor(atom_features, dtype=torch.int64)


In [2]:
res = graph_encoder(batch)

In [7]:
to_dense_adj(
    res.edge_index,
    res.batch,
    res.edge_attr,
).shape

torch.Size([2, 32, 32, 768])

In [4]:
def _prepare_graph_input(
    self,
    x: torch.Tensor,
    edge_matrix: torch.Tensor,
    hop: torch.Tensor,
    mask: Optional[torch.Tensor] = None,
    task_name: Optional[str] = None,
):
    x = self._encode_node(x)
    if self.training and self.perturb_noise != 0.0:
        perturb = torch.empty_like(x).uniform_(-self.perturb_noise, self.perturb_noise)
        x = x + perturb

    # Append Task Token
    x_with_task = torch.zeros((x.shape[0], x.shape[1] + 1, x.shape[2]), dtype=x.dtype, device=x.device)
    task_token_idx = torch.zeros((x.shape[0],), dtype=torch.long, device=x.device)

    x_with_task[:, 1:, :] = x
    if task_name is not None and isinstance(self.task_token, nn.ModuleDict):
        x_with_task[:, 0, :] = self.task_token[task_name](task_token_idx)
    else:
        x_with_task[:, 0, :] = self.task_token(task_token_idx)

    # Mask with task
    if mask is None:
        mask_with_task = None
    else:
        mask_with_task = torch.zeros(
            (mask.shape[0], mask.shape[1] + 1),
            dtype=mask.dtype,
            device=x.device,
        )
        mask_with_task[:, 1:] = mask
        mask_with_task[:, 0] = True

    hop_with_task = torch.zeros(
        (
            hop.shape[0],
            hop.shape[1] + 1,
            hop.shape[2] + 1,
        ),
        dtype=hop.dtype,
        device=hop.device,
    )
    # distance with task
    # max_hop is $\mathcal{P}_\text{far}$
    hop_clamped = hop.clamp(max=self.max_hop)
    hop_with_task[:, 1:, 1:] = hop_clamped
    # extend hop for unreachable # No need
    # hop_with_task[:, 0, 1:] = hop_clamped[:, 0, :]   #! need to check
    # hop_with_task[:, 1:, 0] = hop_clamped[:, :, 0]   #! need to check
    unreachable_mask = hop_with_task == -1
    # set task_distance
    hop_with_task[:, 0, 1:] = self.TASK_DISTANCE
    hop_with_task[:, 1:, 0] = self.TASK_DISTANCE
    # set unreachable_distance
    hop_with_task[unreachable_mask] = self.UNKNOWN_DISTANCE

    # edge matrix with task
    edge_matrix_with_task = torch.zeros(
        (
            edge_matrix.shape[0],
            edge_matrix.shape[1] + 1,
            edge_matrix.shape[2] + 1,
        ),
        dtype=edge_matrix.dtype,
        device=edge_matrix.device,
    )
    edge_matrix_with_task[:, 1:, 1:] = edge_matrix
    edge_matrix_with_task[hop_with_task != 1] = self.NO_EDGE

    # self edge
    edge_matrix_with_task[
        :,
        list(range(edge_matrix_with_task.shape[1])),
        list(range(edge_matrix_with_task.shape[2])),
    ] = self.SELF_EDGE
    edge_matrix_with_task[hop_with_task == self.TASK_DISTANCE] = self.TASK_EDGE
    edge_matrix_with_task[unreachable_mask] = self.UNKNOWN_EDGE

    return x_with_task, edge_matrix_with_task, hop_with_task, mask_with_task

tensor([[ 5, 13,  1,  0,  4,  5,  3,  0,  2,  0,  0],
        [ 5, 13,  1,  0,  4,  5,  0,  0,  2,  0,  0],
        [ 5, 13,  1,  0,  4,  5,  3,  0,  2,  0,  0],
        [ 5, 13,  1,  0,  4,  5,  3,  0,  2,  0,  0],
        [ 7, 15,  1,  0,  2,  5,  0,  0,  1,  0,  0],
        [ 5, 13,  1,  0,  3,  5,  0,  0,  1,  0,  0],
        [ 7, 15,  1,  0,  1,  5,  0,  0,  1,  0,  0],
        [ 6, 14,  1,  0,  3,  5,  0,  0,  1,  0,  1],
        [ 5, 13,  1,  0,  4,  5,  2,  0,  2,  0,  1],
        [ 5, 13,  1,  0,  4,  5,  2,  0,  2,  0,  1],
        [ 6, 14,  1,  0,  3,  5,  0,  0,  2,  0,  1],
        [ 5, 13,  1,  0,  4,  5,  2,  0,  2,  0,  0],
        [ 5, 13,  1,  0,  3,  5,  0,  0,  1,  1,  1],
        [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
        [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
        [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
        [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
        [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
        [ 5, 13,  1,  0,  4,

In [30]:
from torch_geometric.utils import degree
degree(data1.edge_index[0].numpy(), num_nodes = len(data1.x))

AttributeError: 'numpy.ndarray' object has no attribute 'device'

In [27]:
data1.edge_index[0]

tensor([ 0,  1,  1,  2,  1,  3,  1,  4,  4,  5,  5,  6,  5,  7,  7,  8,  8,  9,
         9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 10, 18,
        18, 19, 19, 20, 20, 21, 21, 22, 19,  7, 17, 12])

In [ ]:
num_nodes

In [22]:
batch = to_dense_batch(batch.x, batch.batch, fill_value=0)

In [23]:
batch

(tensor([[[ 5, 13,  1,  0,  4,  5,  3,  0,  2,  0,  0],
          [ 5, 13,  1,  0,  4,  5,  0,  0,  2,  0,  0],
          [ 5, 13,  1,  0,  4,  5,  3,  0,  2,  0,  0],
          [ 5, 13,  1,  0,  4,  5,  3,  0,  2,  0,  0],
          [ 7, 15,  1,  0,  2,  5,  0,  0,  1,  0,  0],
          [ 5, 13,  1,  0,  3,  5,  0,  0,  1,  0,  0],
          [ 7, 15,  1,  0,  1,  5,  0,  0,  1,  0,  0],
          [ 6, 14,  1,  0,  3,  5,  0,  0,  1,  0,  1],
          [ 5, 13,  1,  0,  4,  5,  2,  0,  2,  0,  1],
          [ 5, 13,  1,  0,  4,  5,  2,  0,  2,  0,  1],
          [ 6, 14,  1,  0,  3,  5,  0,  0,  2,  0,  1],
          [ 5, 13,  1,  0,  4,  5,  2,  0,  2,  0,  0],
          [ 5, 13,  1,  0,  3,  5,  0,  0,  1,  1,  1],
          [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
          [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
          [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
          [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  1,  1],
          [ 5, 13,  1,  0,  3,  5,  1,  0,  1,  

In [8]:
from graphomer.model.graphomer import Graphomer
model = Graphomer(d_model=128, num_heads=8, d_ff=128)

In [9]:
from torch_geometric.loader import DataLoader
dl = DataLoader([data1, data2], batch_size=2)
for batch in dl:
    model(batch)

EinopsError:  Error while processing rearrange-reduction pattern "b n (h d) -> b h n d".
 Input tensor shape: torch.Size([55, 128]). Additional info: {'h': 8}.
 Wrong shape: expected 3 dims. Received 2-dim tensor.

In [11]:
to_dense_batch(batch)

AttributeError: 'GlobalStorage' object has no attribute 'device'

In [5]:
model(batch)

EinopsError:  Error while processing rearrange-reduction pattern "b n (h d) -> b h n d".
 Input tensor shape: torch.Size([55, 128]). Additional info: {'h': 8}.
 Wrong shape: expected 3 dims. Received 2-dim tensor.

In [7]:
from graphomer.datamodule.shortest_path import ShortestPathGenerator
st = ShortestPathGenerator(max_num_nodes=128)

In [14]:
atom_features_dim

NameError: name 'atom_features_dim' is not defined

In [9]:
st(data1)

Data(x=[23, 11], edge_index=[2, 48], edge_attr=[48, 3], degree=[23, 1], hop=[128, 128])

In [13]:
(st(data2).hop == st(data1).hop).all()

tensor(False)

In [11]:
from torch_geometric.utils import to_networkx
g = to_networkx(data1)

In [13]:
import networkx as nx
res = nx.all_pairs_shortest_path_length(g)

In [15]:
dict(res)

{0: {0: 0,
  1: 1,
  2: 2,
  3: 2,
  4: 2,
  5: 3,
  6: 4,
  7: 4,
  8: 5,
  19: 5,
  9: 6,
  18: 6,
  20: 6,
  10: 7,
  21: 7,
  11: 8,
  22: 8,
  12: 9,
  13: 10,
  17: 10,
  14: 11,
  16: 11,
  15: 12},
 1: {1: 0,
  0: 1,
  2: 1,
  3: 1,
  4: 1,
  5: 2,
  6: 3,
  7: 3,
  8: 4,
  19: 4,
  9: 5,
  18: 5,
  20: 5,
  10: 6,
  21: 6,
  11: 7,
  22: 7,
  12: 8,
  13: 9,
  17: 9,
  14: 10,
  16: 10,
  15: 11},
 2: {2: 0,
  1: 1,
  0: 2,
  3: 2,
  4: 2,
  5: 3,
  6: 4,
  7: 4,
  8: 5,
  19: 5,
  9: 6,
  18: 6,
  20: 6,
  10: 7,
  21: 7,
  11: 8,
  22: 8,
  12: 9,
  13: 10,
  17: 10,
  14: 11,
  16: 11,
  15: 12},
 3: {3: 0,
  1: 1,
  0: 2,
  2: 2,
  4: 2,
  5: 3,
  6: 4,
  7: 4,
  8: 5,
  19: 5,
  9: 6,
  18: 6,
  20: 6,
  10: 7,
  21: 7,
  11: 8,
  22: 8,
  12: 9,
  13: 10,
  17: 10,
  14: 11,
  16: 11,
  15: 12},
 4: {4: 0,
  1: 1,
  5: 1,
  0: 2,
  2: 2,
  3: 2,
  6: 2,
  7: 2,
  8: 3,
  19: 3,
  9: 4,
  18: 4,
  20: 4,
  10: 5,
  21: 5,
  11: 6,
  22: 6,
  12: 7,
  13: 8,
  17: 8,
  14: